# Demo: Applying Parameters to the CASTable Object

### 1. Import Packages and Connect to the CAS Server

Visit the documentation for the SWAT [(SAS Scripting Wrapper for Analytics Transfer)](https://sassoftware.github.io/python-swat/index.html) package.

In [ ]:
## Import packages
import swat
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Set options
pd.set_option('display.max_columns', None)

## Connect to CAS
conn = swat.CAS('server.demo.sas.com', 30571, 'student', 'Metadata0', name = 'py03d02')

## Function to load the loans_raw.sashdat file into memory if necessary
def loadLoans():
    conn.loadTable(path ='loans_raw.sashdat', caslib = 'PIVY',
                   casOut = {'name' : 'loans_raw',
                            'caslib' : 'casuser',
                            'promote' : True})

### 2. Explore Available CAS Tables and Data Source Files


a. Use the tableInfo action to view all available in-memory tables in the **Casuser** caslib. If the **LOANS_RAW** CAS table is not available, uncomment the loadLoans function and execute the cell.

In [ ]:
#loadLoans()
conn.tableInfo(caslib = 'casuser')

b. Reference the **loans_raw** CAS table in the variable **tbl** using the CASTable method.

In [ ]:
tbl = conn.CASTable('loans_raw', caslib = 'casuser')
tbl

c. Preview the CAS table using the fetch action.

In [ ]:
tbl.fetch(to = 5)

### 3. Working with the CAS Table Reference (Three Methods)

Reference the **CASTable** object and add a question mark. This enables you to see a full listing of possible **CASTable** parameters.

**Note:** You can also access the CAS table parameters in the documentation. Go to an action like the [fetch action](https://go.documentation.sas.com/doc/en/pgmsascdc/v_017/caspg/cas-table-fetch.htm#SAS.cas-table-fetch-table). Scroll down and select the table parameter, and then select the [castable](https://go.documentation.sas.com/doc/en/pgmsascdc/v_017/caspg/compg-castable-742param.htm) parameter link.



In [ ]:
tbl?

#### Method 1 - Object-Oriented Method

a. Print the **tbl** object's type and current value. Notice that **tbl** is a **CASTable** object and it references the **LOANS_RAW** table in the **Casuser** caslib.

In [ ]:
display(type(tbl), tbl)

b. Add the vars parameter to the **CASTable** object to select a list of columns to retrieve from the input CAS table. Execute the print function and fetch action on the **CASTable** object **tbl**. The print function shows that the vars parameter was added to the **CASTable** object, and the fetch action returns the specified columns from the CAS table.

In [ ]:
tbl.vars = ['ID','Salary', 'Category', 'Amount', 'InterestRate']
print(tbl)

tbl.fetch(to = 5)

c. Add the where parameter to subset the input CAS table for all rows where **Category** equals *Credit Card*. Execute the print function and fetch action on the **CASTable** object. The print function shows that the where parameter was added to the **CASTable** object, and fetch action now selects the specified rows and columns.

In [ ]:
tbl.where = 'Category = "Credit Card"'
print(tbl)

tbl.fetch(to = 5)

c. Add the computedVarsProgram parameter to create calculated columns. The computedVarsProgram parameter can
- create multiple columns by separating each assignment statement with a semicolon
- use most [SAS functions](https://go.documentation.sas.com/doc/en/pgmsascdc/v_018/lefunctionsref/titlepage.htm?homeOnFail)
- execute IF/THEN logic to create new columns
- modify the length and label of a column using a SAS LENGTH or LABEL statement

In this example, two columns are created:
- First, a new column named **MonthlySalary** calculates the monthly salary of each customer by dividing **Salary** by 12. 
- The second column is created using SAS IF-THEN logic. If **Salary** is less than 50,000 the new column **Tier** will have a value of *Tier1*. If **Salary** greater than or equal to 50,000, and less than 100,000 the value will be *Tier2*. Otherwise, the value is *Other*. 

Execute the cell. Notice that the new **CASTable** object has the where, vars, and computedVarsProgram parameters.

**Note:** Spanning strings over multiple lines can be done using Python’s triple quotation marks.

In [ ]:
tbl.computedVarsProgram = '''MonthlySalary = round(Salary/12);
                             if Salary < 50000 then Tier = 'Tier1';
                                else if Salary < 100000 then Tier = 'Tier2';
                                else Tier = 'Other';
                          '''
tbl

d. Display the values specified in the vars parameter. Then execute the fetch action on the new **CASTable** object with the new calculated columns. Notice that even though computedVarsProgram was added to the **CASTable** object, new columns are not shown in the results of the fetch action because they were not specified in the vars parameter. The calculation is occurring, but it's not returned to the client in the output.

In [ ]:
## View the value of the vars parameter
display(tbl.vars)

## Execute the fetch action
tbl.fetch(to = 5)

e. Add the new calculated columns **MonthlySalary** and **Tier** to the vars parameter using the extend method. Then view the value of the vars parameter in the **CASTable** object and execute the fetch action to view the calculated columns in the results.

In [ ]:
tbl.vars.extend(['MonthlySalary','Tier'])
display(tbl.vars)

tbl.fetch(to = 5)

f. The **CASTable** object can also be used with other actions or methods. For example, you can now use the **tbl** object with the head method for similar results. 

In [ ]:
tbl.head()

g. You can also use an alternate method by adding parameters on the params level. You specify the **CASTable** object **tbl2**, params, and then the parameters to add. Using this method avoids any name collisions that can occur. The cell below produces identical results as the above cells.

In [ ]:
## Create a new CASTable reference
tbl2 = conn.CASTable('loans_raw', caslib = 'casuser')

## Add parameters on the params level
tbl2.params.where = 'Category="Credit Card"'
tbl2.params.vars = ['ID','Salary','Category', 'Amount', 'InterestRate','MonthlySalary','Tier']
tbl2.params.computedVarsProgram = '''MonthlySalary = round(Salary/12);
                                     if Salary < 50000 then Tier = 'Tier1';
                                       else if Salary < 100000 then Tier = 'Tier2';
                                       else Tier = 'Other';
                                  '''
display(tbl2)

tbl2.head(5)

h. You can delete individual parameters using the keyword del, and then the **CASTable** object **tbl** and the parameter to delete. Then print the **CASTable** object **tbl** and notice that the where, vars, and computedVarsProgram parameters were removed.

In [ ]:
## Delete parameters
del tbl.vars
del tbl.where
del tbl.computedVarsProgram

## View the CASTable object
print(tbl)

## Alternate method to delete multiple parameters in a single step
#tbl.del_params('vars', 'where', 'computedvarsprogram')

#### Method 2 - Statically Typed

You can achieve the same results by adding all the parameters within the table parameter in an action as a dictionary. This method is less flexible than the object-oriented method because you can use this input CAS table only in this action.

In [ ]:
conn.fetch(table = {'name' : 'loans_raw',
                    'caslib' : 'casuser',
                    'vars' : ['ID','Salary','MonthlySalary','Tier','Category', 'Amount', 'InterestRate'],
                    'where' : 'Category="Credit Card"',
                    'computedVarsProgram' :  '''MonthlySalary  =round(Salary/12);
                                                if Salary < 50000 then Tier='Tier1';
                                                   else if Salary < 100000 then Tier='Tier2';
                                                   else Tier = 'Other';
                                             '''},
           to = 5)

#### Method 3 - Using Variables

a. You can also create variables to use in actions. Here, the **monthlySalary** variable is created to hold the string to create the **MonthlySalary** column, and the **tier** variable contains the calculation for the **Tier** column. Then the **input_tbl** variable is created to store a dictionary with information about the input CAS table using the name, caslib, vars, where, and computedVarsProgram parameters. 

Once the variables are created, you can add them within an action. In this example, the table parameter specifies the **input_tbl** variable. The results are the same as the previous two approaches.

In [ ]:
## Specify the computedVarsProgram calculation statements
monthlySalary = 'MonthlySalary = round(Salary/12);'
tier = '''
        if Salary < 50000 then Tier = 'Tier1';
          else if Salary < 100000 then Tier = 'Tier2';
          else Tier = 'Other';
       '''

## Specify the table parameter dictionary values
input_tbl = {'name' : 'loans_raw',
             'caslib' : 'casuser',
             'vars' : ['ID','Salary','MonthlySalary','Tier','Category', 'Amount', 'InterestRate'],
             'where' : 'Category = "Credit Card"',
             'computedVarsProgram' :  monthlySalary + tier}

conn.fetch(table = input_tbl, to = 5)

b. You can also use the Python dict function to create a dictionary using keyword arguments. It avoids having to enclose dictionary key names in quotation marks.

In [ ]:
## Alternate using the dict function
input_tbl = dict(name = 'loans_raw', 
                 caslib = 'casuser', 
                 vars = ['ID','Salary','MonthlySalary','Tier','Category', 'Amount', 'InterestRate'], 
                 where = 'Category = "Credit Card"', 
                 computedVarsProgram = monthlySalary + tier)

conn.fetch(table = input_tbl, to = 5)

### 4. Terminate the CAS Session

It's best practice to always terminate the CAS session when you are done.

In [ ]:
conn.terminate()